# Sub-Agents: Dynamic Creation

In this pattern, the coordinator dynamically creates sub-agents at runtime. There are no pre-defined specialists. The coordinator decides what kind of agent it needs, describes its role, and spawns it on demand.

In [ ]:
from pydantic_ai import RunContext
from agentic_patterns.core.agents import get_agent, run_agent

## The Generic Sub-Agent Tool

A single tool that creates and runs any sub-agent. The coordinator specifies the system prompt and task.

In [ ]:
async def run_sub_agent(ctx: RunContext[None], system_prompt: str, task: str) -> str:
    """Create and run a sub-agent with the given system prompt to perform the task."""
    print(f"[Creating sub-agent] {system_prompt[:50]}...")
    
    sub_agent = get_agent(system_prompt=system_prompt)
    agent_run, _ = await run_agent(sub_agent, task)
    ctx.usage.incr(agent_run.result.usage())
    
    print(f"[Sub-agent completed]")
    return agent_run.result.output

## The Coordinator

The coordinator has only one tool but can create any specialist it needs.

In [ ]:
coordinator = get_agent(
    tools=[run_sub_agent],
    system_prompt="""You are a problem-solving coordinator. For complex tasks, break 
them down and delegate to specialized sub-agents using the run_sub_agent tool.

When using run_sub_agent:
- system_prompt: Define the sub-agent's expertise (e.g., "You are a financial analyst...")
- task: The specific question or task for that sub-agent

You can create any specialist you need. Synthesize their outputs into a final answer."""
)

## Example: Multi-Domain Analysis

The coordinator decides what specialists it needs and creates them dynamically.

In [ ]:
agent_run, _ = await run_agent(
    coordinator,
    """I'm considering investing in a small coffee shop. The initial investment is $80,000, 
    expected monthly revenue is $15,000, and monthly costs are around $12,000. The lease 
    is for 3 years. Should I do it?""",
    verbose=True
)
print("\n" + "="*60)
print("FINAL RESPONSE:")
print("="*60)
print(agent_run.result.output)

In [ ]:
usage = agent_run.result.usage()
print(f"Total tokens: {usage.total_tokens}")